# FPP optimization workflow 

In [1]:
using Revise
using FUSE
using IMAS
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Debug);

## Make new FPP equilibirum from scratch

### Initialize from FPP v1_demount

In [ ]:
dd = IMAS.dd()
ini, act = FUSE.case_parameters(:ITER, init_from=:scalars);

### Modify ini

In [ ]:
ini.nbi.power_launched = 20e6
ini.ec_launchers.power_launched = 30e6
ini.ic_antennas.power_launched = 0e6

ini

In [ ]:
act.ActorECsimple.rho_0 = 0.0

act

### Modify act

In [ ]:
act.ActorEquilibrium.model = :TEQUILA
act.ActorTEQUILA.free_boundary = false

act.ActorHFSsizing.do_plot = true
act.ActorHFSsizing.verbose = true
act.ActorHFSsizing.error_on_technology = false
act.ActorHFSsizing.error_on_performance = false

act.ActorPFcoilsOpt.do_plot = false
act.ActorPFcoilsOpt.update_equilibrium = false
act.ActorPFcoilsOpt.optimization_scheme = :none


act.ActorCoreTransport.model = :FluxMatcher
act.ActorFluxMatcher.do_plot = false
act.ActorFluxMatcher.verbose = false

act.ActorStationaryPlasma.convergence_error = 1e-2
act.ActorStationaryPlasma.max_iter = 5
act.ActorStationaryPlasma.do_plot = true
act.ActorStationaryPlasma.verbose = true

act.ActorFluxMatcher


## Run WholeFacility Actor

In [ ]:
dd = IMAS.dd()
@time FUSE.init(dd, ini, act; do_plot=false);
display(IMAS.extract(dd))

In [ ]:
@time FUSE.ActorHFSsizing(dd, act);

## Summary

In [ ]:
#redirect_stdio(stdout="timing.txt") do
#    show(FUSE.timer)
#end
show(FUSE.timer)

In [ ]:
display(IMAS.extract(dd))
FUSE.digest(dd)